# Metrics

In [ ]:
% matplotlib inline

import numpy as np
import pandas as pd
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import cv2

from mymods.lauthom import *

import sys
sys.path.append('./pycode')




### Clipping min/max

In [ ]:

img = np.array([0,1,2,3,4,5,6,7,8,9,20])

norm = (img - img.mean()) / img.std()
norm
wt = 2
img = np.clip(img, -wt*img.std(), wt*img.std())
(img - img.mean()) / img.std()

### Metrics

In [ ]:
true = (np.random.normal(-1, 1, 100) > 0)//1
pred = (np.random.normal(-1, 1, 100) > 0)//1
H, _, _ = np.histogram2d(true, pred, bins=(2,2))
H, true, pred

np.sum(true), np.sum(pred)
TN, FN, FP, TP = np.split(H.ravel(), 4)
TN, FN, FP, TP

# IuO
TP / max(1e-9, (FN + FP + TP))


# precision
TP / np.sum(pred)

# recall
TP / np.sum(true)

# TP
# F

In [ ]:
def IoU(true, pred):
    """IoU"""
    H, _, _ = np.histogram2d(true, pred, bins=(2,2))
    TN, FN, FP, TP = np.split(H.ravel(), 4)
    if TP==0:
        return 0
    return TP / max(1e-9, (FN + FP + TP))

In [ ]:
%%timeit
IoU(true, pred)

In [ ]:
def IoU2(true, pred):
    """IoU"""
    H, _, _ = np.histogram2d(true, pred, bins=(2,2))
    if H[1,1]==0:
        return 0
    return H[1,1] / (H[1,0] + H[0,1] + H[1,1])

In [ ]:
%%timeit
IoU2(true, pred)

In [ ]:
%%timeit
_iou(true, pred) # 25x faster

In [ ]:
# truth = np.array([[1,1,1,1,0],[0,0,0,0,1],[0,0,0,0,1]])
# pred = np.array([[1,1,1,0,0],[1,1,0,0,1],[0,0,0,0,1]])
H, xedges, yedges = np.histogram2d(truth.ravel(), pred.ravel(), bins=(2,2))
H

In [ ]:
TN, FN, FP, TP = np.split(H.ravel(), 4)
TP / max(1e-9, (FN + FP + TP))
TN, FN, FP, TP

In [ ]:
def iou(true, pred):
    """"""
    CM, _, _ = np.histogram2d(true.ravel(), pred.ravel(), bins=(2,2))
    TN, FN, FP, TP = np.split(CM.ravel(), 4)
    return float(TP / max(1e-9, (FN + FP + TP)))

In [ ]:
iou(truth, pred) == 5/8

In [ ]:
intersection = np.sum(truth * pred)
intersection

union = np.sum((truth + pred) > 0)
union

In [ ]:
def bool_indices(mask):
    """Convert True/False list to list of True indices"""
    import pandas as pd
    df = pd.DataFrame(np.arange(len(mask)))
    return [i for i in df.index[mask]]

[bool_indices([True, False, True, False, True])]

# [99, 101, 43, 51][bool_indices([True, False, True, False, True])]

In [ ]:
bool_indices([True, False, True, False, True]).squeeze()

In [ ]:
[0, 1, 2]

In [ ]:
def bool_indices2(mask, series):
    """Convert True/False list to list of True indices"""
    from itertools import compress
    return series[list(compress(list(range(len(mask))), mask))]


In [ ]:
m = [True, False, True, False, True]
bool_indices2(m, [99, 101, 43, 51])

In [ ]:
m = [True, False, True, False, True]
[i for i, x in enumerate(range(len(m))) if x]

In [ ]:
l = [99, 101, 43, 51, 0, 1]
m = [True, False, True, False, True]

def bool_indices(mask, obj=None):
    """Convert True/False list to list of True indices"""
    if obj is None:
        return np.arange(len(m))[m].tolist()
    try:
        return np.array(obj)[m].tolist()
    except:
        print('Object to index length does not match the mask')

bool_indices(m)

In [ ]:
np.sum([0, 2, 4])

In [ ]:
import cv2
cv2.resize(np.zeros(2000), (100,100), interpolation=cv2.INTER_CUBIC)

In [ ]:
cv2.resize(np.zeros(10), (101, 101))

In [ ]:
def _iou_metric(labels, y_pred, print_table=False):
    """"""

    # Confusion matrix
    intersection, _, _ = np.histogram2d(labels.flatten(), y_pred.flatten(), bins=(2, 2)) #[0] - confusion matrix

    # Compute areas (needed for finding the union between all objects)
    area_true, _ = np.histogram(labels, bins=2) # ones
    area_pred, _ = np.histogram(y_pred, bins=2) # ones
    area_true = np.expand_dims(area_true, -1)   # column vector [ones, zeros].T
    area_pred = np.expand_dims(area_pred, 0)    # row vector [zeros, ones]
    # row vector + column vector = broadcasted matrix

    # Compute union = CM - CM = CM
    union = area_true + area_pred - intersection

    # Exclude background(TN) from the analysis - inter section is CM 
    intersection = intersection[1:, 1:]         # get true positives from intersection CM
    union = union[1:, 1:]                       # get true positives from union CM
    union[union == 0] = 1e-9                    # do not devide by zero

    # Compute the intersection over union (float)
    iou = intersection / union

    # Precision helper function
    def precision_at(threshold, iou):
        mask = iou > threshold
        # Image is
        true_pos = np.sum(mask, axis=1) == 1   # Correct objects
        false_pos = np.sum(mask, axis=0) == 0  # Missed objects
        false_neg = np.sum(mask, axis=1) == 0  # Extra objects
        return np.sum(true_pos) / (np.sum(true_pos) + np.sum(false_pos) + np.sum(false_neg) + 1e-9)

    # Loop over IoU thresholds
    miou = np.mean([precision_at(t, iou) for t in np.arange(0.5, 1.0, 0.05)])
    return miou

In [ ]:
y_true = np.zeros(128*128).reshape(128, 128)
y_true[:, 0] = 1
y_true = np.stack((y_true, y_true))
print(y_true, np.sum(y_true))

area_true = np.histogram(y_pred, bins=2)[0]
area_true = np.expand_dims(area_pred, -1)
area_true

In [ ]:
y_pred = np.ones(128*128).reshape(128, 128)
y_pred[0, :] = 0
y_pred = np.stack((y_pred, y_pred))
print(y_pred, np.sum(y_pred))

area_pred = np.histogram(y_pred, bins=2)[0]
area_pred = np.expand_dims(area_pred, 0)
area_pred

In [ ]:
CM_area = (area_true + area_pred).astype(int)[0]
CM_area

In [ ]:
CM_intersection, _, _ = np.histogram2d(y_true.flatten(), y_pred.flatten(), bins=(2, 2))
CM_intersection = CM_intersection.astype(int)
CM_intersection

In [ ]:
CM_intersection[1:, 1:]

In [ ]:
CM_union = CM_area - CM_intersection
CM_union = CM_union.astype(int)
CM_union

In [ ]:
CM_union[1:, 1:]

In [ ]:
CM_union[union==0] = 1e-9  

In [ ]:
IOU = CM_intersection[1:, 1:] / CM_union[1:, 1:]
IOU

In [ ]:
def precision_at(threshold, iou):
    mask = iou > threshold
    # Image is
    true_pos = np.sum(mask, axis=1) == 1   # Correct objects
    false_pos = np.sum(mask, axis=0) == 0  # Missed objects
    false_neg = np.sum(mask, axis=1) == 0  # Extra objects
    return np.sum(true_pos) / (np.sum(true_pos) + np.sum(false_pos) + np.sum(false_neg) + 1e-9)

In [ ]:
precision_at(.0005, IOU)

In [ ]:
m = IOU <.0005; m
np.sum(m, axis=1) == 1
np.sum(m, axis=0) == 0
np.sum(m, axis=1) == 0

In [ ]:
np.sum(y_pred * y_true)

In [ ]:
np.sum((y_pred + y_true) > 0)

In [ ]:
def iou(y_true, y_pred):
    """"""
    CM, _, _ = np.histogram2d(y_true.ravel(), y_pred.ravel(), bins=(2,2))
    TN, FN, FP, TP = np.split(CM.ravel(), 4)
    return float(TP / max(1e-9, (FN + FP + TP)))

In [ ]:
iou(y_true, y_pred)

In [ ]:
np.sum(y_pred * y_true) / np.sum((y_pred + y_true) > 0)

In [ ]:
TN, FN

In [ ]:
def iou_metric(y_true_in, y_pred_in, print_table=False):
    labels = y_true_in
    y_pred = y_pred_in
    
    true_objects = 2
    pred_objects = 2

    intersection = np.histogram2d(labels.flatten(), y_pred.flatten(), bins=(true_objects, pred_objects))[0]

    # Compute areas (needed for finding the union between all objects)
    area_true = np.histogram(labels, bins = true_objects)[0]
    area_pred = np.histogram(y_pred, bins = pred_objects)[0]
    area_true = np.expand_dims(area_true, -1)
    area_pred = np.expand_dims(area_pred, 0)

    # Compute union
    union = area_true + area_pred - intersection

    # Exclude background from the analysis
    intersection = intersection[1:,1:]
    union = union[1:,1:]
    union[union == 0] = 1e-9

    # Compute the intersection over union
    iou = intersection / union

    # Precision helper function
    def precision_at(threshold, iou):
        matches = iou > threshold
        true_positives = np.sum(matches, axis=1) == 1   # Correct objects
        false_positives = np.sum(matches, axis=0) == 0  # Missed objects
        false_negatives = np.sum(matches, axis=1) == 0  # Extra objects
        tp, fp, fn = np.sum(true_positives), np.sum(false_positives), np.sum(false_negatives)
        return tp, fp, fn

    # Loop over IoU thresholds
    prec = []
    if print_table:
        print("Thresh\tTP\tFP\tFN\tPrec.")
    for t in np.arange(0.5, 1.0, 0.05):
        tp, fp, fn = precision_at(t, iou)
        if (tp + fp + fn) > 0:
            p = tp / (tp + fp + fn)
        else:
            p = 0
        if print_table:
            print("{:1.3f}\t{}\t{}\t{}\t{:1.3f}".format(t, tp, fp, fn, p))
        prec.append(p)
    
    if print_table:
        print("AP\t-\t-\t-\t{:1.3f}".format(np.mean(prec)))
    return np.mean(prec)

In [ ]:
iou_metric(y_true, y_true[0,0]==1, True)